In [4]:
# importing libraries
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
db_password=os.getenv("db_password")

In [5]:
# creating object of GooglePalm() class as llm
llm=GooglePalm(
    google_api_key=GOOGLE_API_KEY,
    temperature=0.4
)

In [6]:
# importing librariess for MYSQL database
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [7]:
# for database dependencies
db_user = "root"
db_password = db_password
db_host = "localhost"
db_name = "TIMS"

In [8]:
# calling SQLDatabase
db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
    sample_rows_in_table_info=3
    )

In [ ]:
# table info
print(db.table_info)

In [ ]:
# creating object of SQLDatabaseChain
sql_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
Q_01 = sql_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")

In [ ]:
# Query-02 : The result is wrong
Q_02 = sql_chain.run("How much is the price of the inventory for all small size t-shirts?")

In [ ]:
# the actual query is
# SELECT sum(tims.product.price*tims.product.stock_quantity)
# FROM tims.product WHERE (tims.product.size='S')
Q_02 = sql_chain.run(
    "SELECT sum(tims.product.price*tims.product.stock_quantity) FROM tims.product WHERE (tims.product.size='S')"
    )


In [ ]:
# Query-03  shows error
Q_03 = sql_chain.run(
    "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store"
    )

In [ ]:
# Query-03: The Actual Query
sql_query="""SELECT sum((tims.product.price*tims.product.stock_quantity)-((tims.product.price*tims.product.stock_quantity)*(tims.discounts.pct_discount/100)))
FROM tims.product INNER JOIN tims.discounts ON
tims.product.pid=tims.discounts.pid
WHERE (tims.product.brand='Levi')"""
Q_03 = sql_chain.run(sql_query)

In [ ]:
# Query-04
Q_04 = sql_chain.run("SELECT SUM(price * stock_quantity) FROM product WHERE brand = 'Levi'")

In [ ]:
# Query-05 :it shows wrong answer
Q_05 = sql_chain.run("How many white color Levi's t shirts we have available?")

In [ ]:
# Query-05 :it shows right answer
Q_05 = sql_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")

#### : Few shot learning
- We are going to use few shot learning to fix issues we have gone through

In [16]:
# declaring few shot learning through defining a list
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : Q_01
     },
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': Q_02
     },
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum((tims.product.price*tims.product.stock_quantity)-((tims.product.price*tims.product.stock_quantity)*(tims.discounts.pct_discount/100)))
      FROM tims.product INNER JOIN tims.discounts ON
      tims.product.pid=tims.discounts.pid
      WHERE (tims.product.brand='Levi')""",
     'SQLResult': "Result of the SQL query",
     'Answer': Q_03
     },
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : Q_04
      },
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : Q_05
     }
]

##### creating semantic similarity based on example selector
- create embedding on top of the few_shots
- Store the embeddings into  ChromaDB
- Retrieve the top most semantically close example from the vector store

In [17]:
# importing libraries
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [18]:
# creating and calling
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
to_vectorize = [" ".join(example.values()) for example in few_shots]

c:\Users\mehed\.conda\envs\myenv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\mehed\.conda\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mehed\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 43",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 24023",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum((tims.product.price*tims.product.stock_quantity)-((tims.product.price*tims.product.stock_quantity)*(tims.discounts.pct_discount/100)))\n      FROM tims.product INNER JOIN tims.discounts ON\n      tims.product.pid=tims.discounts.pid\n      WHERE (tims.product.brand='Levi') Result of the SQL query 2813.4",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'

In [20]:
# calling Chroma
vectorstore = Chroma.from_texts(
    to_vectorize, 
    embeddings, 
    metadatas=few_shots
    )

In [21]:
# creating object of SemanticSimilarityExampleSelector() class 
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '43',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '27696',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [22]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [24]:
# importing the libraries
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [25]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


##### Setting up PromptTemplete using input variables

In [26]:
from langchain.prompts.prompt import PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [27]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [28]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [29]:
# new chain created
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [30]:
# Query
new_chain.run("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT stock_quantity FROM product WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(67,), (73,), (61,), (76,)]
Answer:67
> Finished chain.


'67'

In [ ]:
new_chain.run("How much is the price of the inventory for all small size t-shirts?")
new_chain("How much is the price of all white color levi t shirts?")
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')